# Initial census tract data in geopandas

In [1]:
import geoplot as gplt
import geopandas as gpd
import geoplot.crs as gcrs
import imageio
import folium
import requests
import pathlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mapclassify as mc

%matplotlib inline
pd.options.display.max_rows = 999

In [2]:
census_tract = gpd.read_file('./data/census_tract_2010_cook_county/tl_2010_17031_tract10.shp')
census_tract.head()

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,17,031,840300,17031840300,8403,Census Tract 8403,G5020,S,831224,0,+41.8320943,-087.6818822,"POLYGON ((-87.68608 41.82296, -87.68607 41.823..."
1,17,031,840200,17031840200,8402,Census Tract 8402,G5020,S,831087,0,+41.8445748,-087.6491915,"POLYGON ((-87.64633 41.84009, -87.64632 41.839..."
2,17,031,841100,17031841100,8411,Census Tract 8411,G5020,S,1143247,0,+41.8510058,-087.6350978,"POLYGON ((-87.62935 41.85280, -87.62934 41.852..."
3,17,031,841200,17031841200,8412,Census Tract 8412,G5020,S,625405,0,+41.8555618,-087.6833420,"POLYGON ((-87.68814 41.85569, -87.68816 41.856..."
4,17,031,838200,17031838200,8382,Census Tract 8382,G5020,S,1157564,0,+41.8704157,-087.6750794,"POLYGON ((-87.66782 41.87418, -87.66768 41.874..."


In [3]:
census_tract.shape

(1319, 13)

In [4]:
type(census_tract['GEOID10'][0])

str

In [5]:
census_tract['GEOID10'].isna().sum()

0

The original food access data is too large to host on the GitHub repo. Click here to access [the Food Access Research Atlas Data Download 2019](https://www.ers.usda.gov/data-products/food-access-research-atlas/download-the-data/). 

In [6]:
food_access = pd.read_csv('./food_access/FoodAccessResearchAtlasData2019.csv')
food_access.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga County,1,1912,693,0,0.0,0.00,0,...,221.0,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0
1,1001020200,Alabama,Autauga County,1,2170,743,0,181.0,8.34,1,...,214.0,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0
2,1001020300,Alabama,Autauga County,1,3373,1256,0,0.0,0.00,0,...,439.0,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0
3,1001020400,Alabama,Autauga County,1,4386,1722,0,0.0,0.00,0,...,904.0,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0
4,1001020500,Alabama,Autauga County,1,10766,4082,0,181.0,1.68,0,...,1126.0,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0


In [7]:
type(food_access['CensusTract'][0])

numpy.int64

In [8]:
food_access['CensusTract'].isna().sum()

0

In [9]:
food_access['CensusTract'][0]

1001020100

In [10]:
# convert food_access['CensusTract'] to a string so the tables can be merged
food_access['CensusTract']= [str(ct_num) for ct_num in food_access['CensusTract']]

In [11]:
type(food_access['CensusTract'][0])

str

In [12]:
food_access['CensusTract'][0]

'1001020100'

In [13]:
food_access.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga County,1,1912,693,0,0.0,0.00,0,...,221.0,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0
1,1001020200,Alabama,Autauga County,1,2170,743,0,181.0,8.34,1,...,214.0,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0
2,1001020300,Alabama,Autauga County,1,3373,1256,0,0.0,0.00,0,...,439.0,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0
3,1001020400,Alabama,Autauga County,1,4386,1722,0,0.0,0.00,0,...,904.0,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0
4,1001020500,Alabama,Autauga County,1,10766,4082,0,181.0,1.68,0,...,1126.0,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0


In [14]:
food_access[food_access['County'] == 'Cook County'].shape

(1320, 147)

### Merge census_tract and food_access on CensusTract and GEOID10 including only the data that exist in both files.

In [15]:
food_data = food_access.merge(census_tract, left_on = 'CensusTract', right_on = 'GEOID10', how = 'inner')
food_data.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,17031010100,Illinois,Cook County,1,4854,2302,0,218.0,4.49,0,...,17031010100,101,Census Tract 101,G5020,S,381061,0,+42.0212617,-087.6698442,"POLYGON ((-87.66368 42.01940, -87.66384 42.019..."
1,17031010201,Illinois,Cook County,1,6450,2463,0,163.0,2.53,0,...,17031010201,102.01,Census Tract 102.01,G5020,S,504470,0,+42.0160076,-087.6801485,"POLYGON ((-87.68009 42.01254, -87.68027 42.012..."
2,17031010202,Illinois,Cook County,1,2818,1115,0,315.0,11.18,0,...,17031010202,102.02,Census Tract 102.02,G5020,S,351716,0,+42.0160503,-087.6733216,"POLYGON ((-87.67336 42.01937, -87.67311 42.019..."
3,17031010300,Illinois,Cook County,1,6236,2826,0,791.0,12.68,0,...,17031010300,103,Census Tract 103,G5020,S,472017,0,+42.0159407,-087.6665351,"POLYGON ((-87.66506 42.01280, -87.66543 42.012..."
4,17031010400,Illinois,Cook County,1,5042,2098,0,1349.0,26.76,0,...,17031010400,104,Census Tract 104,G5020,S,548754,602605,+42.0064028,-087.6571350,"POLYGON ((-87.65080 41.99848, -87.65500 41.998..."


In [17]:
food_data.shape

(1314, 160)

#### census tract has 1319 rows, food_access has 1320 rows. When merged food_data has 1314 rows.

In [18]:
food_data.isna().sum().sort_values(ascending = False).head()

lasnap20share    1314
laasian10        1314
lasnap10         1314
lahunv10share    1314
lahisp10share    1314
dtype: int64

We see that several features have nulls, and some are entirely null.  We will examine these more later.

In [19]:
food_data.describe().T.head()

,count,mean,std,min,25%,50%,75%,max
Urban,1314.0,0.998478,0.038999,0.0,1.0,1.0,1.00,1.0
Pop2010,1314.0,3953.329528,1788.800486,237.0,2548.0,3848.0,5149.50,16735.0
OHU2010,1314.0,1496.465753,761.006299,100.0,964.5,1424.0,1893.00,9751.0
GroupQuartersFlag,1314.0,0.003805,0.061592,0.0,0.0,0.0,0.00,1.0
NUMGQTRS,1314.0,68.707763,352.577133,0.0,0.0,2.0,30.75,10273.0


Summary statistics for the food access data.  Only the head is currently shown to avoid a very large output cell.

In [19]:
# drop the geometry column for now and then write data to csv
food_data.drop(columns = 'geometry').to_csv('./data/food_data.csv')